# 0.기본 셋팅

In [1]:

import math
import numpy as np
from numpy import linalg as LA
from scipy.sparse import coo_matrix
import pandas as pd
from IPython.display import HTML, display

In [2]:
np.set_printoptions(precision=5)
pd.set_option('display.precision', 5)

In [3]:

movies = pd.read_csv('../src/movies_w_imgurl.csv')

In [4]:
ratings = pd.read_csv('../src/ratings-9_1.csv')
train = ratings[ratings['type'] == 'train'][['userId', 'movieId', 'rating']]
test = ratings[ratings['type'] == 'test'][['userId', 'movieId', 'rating']]

In [5]:
movieGenres = pd.DataFrame(
    data=movies['genres']  
    .str
    .split('|')   
    .apply(pd.Series)
    .stack() # 정리하기 편하게 stack으로 쌓아준다.
    ,columns=['genre'])
movieGenres.index = movieGenres.index.droplevel(1)

In [6]:
genres = pd.DataFrame(data = movieGenres.groupby('genre')['genre'].count())
genres.columns= ['movieCount']
totalItems = movies.shape[0]
genres['idf'] = genres['movieCount'].apply(lambda x: math.log10(totalItems/x))

In [7]:
movieGenreWeights = movieGenres.join(genres['idf'], on='genre')
movieWeights = movies[['movieId']]

In [8]:

for genre in genres.index:
    movieWeights = movieWeights\
    .join(movieGenreWeights[movieGenreWeights['genre'] == genre]
    [['idf']].rename(columns={'idf':genre})) 
    movieWeights.fillna(0, inplace=True)


* 아래는 sims를 구하는 함수이다.

# 1.평점기준.

* 유저별로 평점기준이 다를수있기에. 각 유저의 평점점수폭을 맞춰준다.


# 2.평과 방법
* 그룹내의 유저의 합으로 추천된 결과가,  각각의 유저별로 추천된 영화등수 의 등수를 평규낸다.
* play(userlist,train,testsims)
* userlist 는 유저집단이다.
* train은 과거기록이다.
* testsims는 사용될 유사도이다.ex) movie_sims


In [27]:

def users(userId,train,testsims):

    userRatings = train[train['userId'] == userId][['movieId', 'rating']]

    recSimSums = testsims.loc[userRatings['movieId'].values, :].sum().values
    recWeightedRatingSums = np.matmul(testsims.loc[userRatings['movieId'].values, :].T.values, userRatings['rating'].values)
    recItemRatings = pd.DataFrame(data=np.divide(recWeightedRatingSums, recSimSums), index=testsims.index)
    recItemRatings.columns = ['prediction']   
    
    
    predictionAvg = recItemRatings['prediction'].sum()/len(recItemRatings['prediction'])
    recItemRatings['prediction']=recItemRatings/predictionAvg
    
    count=0
    _list=[]
    for i in recItemRatings['prediction'].notnull():
        
        index=recItemRatings.index[count]
        if i==False:
            recItemRatings['prediction'][index]=0
        count+=1

    return recItemRatings*2.5



In [28]:
def multiRec(user_number_list,train,testsims):
    multi=0
    for i in user_number_list:
        multi += users(i,train,testsims)
    multi/=len(user_number_list)
    multi=multi.sort_values(by='prediction', ascending=False)
    return multi


In [29]:
def check(user_number_list,num,train,testsims):
    _list=[]
    for j in user_number_list:
         _list.append(want_test(j,num,train,testsims))# User 99 => 400등,  13=> 30등,  14= 1500등
    res=float(sum(_list))/len(_list)
    return res

In [30]:
def play(user_number_list,train,testsims):
    for i in multiRec(user_number_list,train,testsims).index[0:5]:
        print "영화 " ,i
        a=check(user_number_list,i,train,testsims)
        print a,"= 평균등수."

# 3. 방법은 총 3가지
* 방법1 유저 평점평균 기반
* 방법2 유저 취향 합 기반
* 방법3 유저 유사도 기반

# 방법1. 유저 평점평균 기반 추천방법

* 원리
* 입력받은 유저리스트의 유저들 각각의 영화별 예측 평점을 구한뒤. 전체 유저들을 합친 영화별 평점 평균을 구한다.
* 모든 유저가 무난하게 좋아할만한 영화들을 고를수있다. 가장 무난한방법.


* 단점
* 모두가 좋아하는 것을 맞추기는 어렵다.

In [31]:
movieNorms = pd.DataFrame(
    data=LA.norm(movieWeights
                 .iloc[:,1:] 
                 .values, 
                ord=2, axis=1),

    columns=['norm2'])    

In [32]:
normalizedMovieWeights = pd.DataFrame(index=movieWeights.index) #빈 데이타 프레임만들기
norms = movieNorms['norm2']
for genre in genres.index:
    normalizedMovieWeights[genre] = movieWeights[genre].divide(norms)
    
movie_sims = pd.DataFrame(data=np.matmul(normalizedMovieWeights, normalizedMovieWeights.T), index=movieWeights['movieId'])
movie_sims.columns = index=movieWeights['movieId']

# 방식 2. 유저 취향합 기반 추천방법

* A,B,C,D를 한 그룹으로 생각후 Alist+Blist+Clist+Dlist를 가지고  영화추천받기.
* 즉, 유저A,B,C,D가 합쳐진 새로운 유저 X를 만든다.

* 원리
* 입력받은 유저리스트의 유저들 각각의 영화 기록을 전부모아서. 영화를 추천받는다.

* 단점
* 유저마다 영화 기록이 다르고 그로인한 선로 장르가 다르지만. 이를 전부 합쳐서 영화를 추천받기에.
* 전혀 보지않았던 장르가 추천될 경우가 존재.

In [33]:
user_number_list=[11,12,13,14]

In [34]:
count=0
_list=[]
for i in train['userId']:
    index=train.index[count]
    if i in user_number_list:
        _list.append(["userX",train['movieId'][index] , train['rating'][index]])
    count+=1
    
df=pd.DataFrame(_list,columns=['userId','movieId','rating'])

In [35]:
train2=train.append(df)

In [36]:

multiRec(["userX"],train2,movie_sims)
check(user_number_list,3339,train2,movie_sims)

/Users/ikku_study/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:8: RuntimeWarning: invalid value encountered in divide


유저 11 의 영화 3339 의 평점은 = 2.81994100433
전체 9125 중에 333 등
유저 12 의 영화 3339 의 평점은 = 0.0
전체 9125 중에 9125 등
유저 13 의 영화 3339 의 평점은 = 3.07439049877
전체 9125 중에 25 등
유저 14 의 영화 3339 의 평점은 = 0.0
전체 9125 중에 9125 등


4652.0

# 방법3. 유저 유사도 기반 추천방법


* 원리
* 입력받은 유저리스트의 유저들을 유사도를 맞춰 그룹을 짓는다.
* 이후 그룹별로 선호 영화를 뽑는다.

* 단점
* A그룹이 좋아하는 영화가 B는 싫어하는 영화가 될수있고 C는 보통인 영화가 될수있다.

In [16]:
count=0
idxcheck=0
taste=pd.DataFrame()
genres_df=pd.DataFrame()
moviedict={}

for idx,value in movieWeights['movieId'].iteritems():
    moviedict[value]=idx # value가  movieId이고 idx는 index

for userId in range(1,int(train.ix[train.index[-1]]["userId"])):
    
    for i in train.index[idxcheck:]:
       
        movieId=train['movieId'][i]
        #print i,userId,movieId
        if train['userId'][i] ==userId:
    
            _list=train['rating'][i]*movieWeights.ix[moviedict[movieId]]
            genres_df.insert(0,'d',_list,1)
            # 위 insert방식은 dataframe 이 한번 만들어지고 그 dataframe을 밑에 taste에 더하게되서 시간소비가 걸림.
            # 이거대신 바로 평균을내고 그 평균으로 taste를 만들면 편할것.
            
        else:
            #print "error"
            #print genres_df
            genres_df=genres_df.drop('movieId')
            genres_df=genres_df.mean(axis=1)
            taste.insert(userId-1,userId,genres_df.T,1)
            #print taste
            genres_df=pd.DataFrame()
            
            break
        idxcheck+=1
        #if count==20:
        #    break
        count+=1
taste=taste.T

* 아래는 유저별 유사도를 구하는 Cosine similarity이다.

In [17]:
tasteNorms = pd.DataFrame(
    data=LA.norm(taste
                 .iloc[:,:] 
                 .values, 
                ord=2, axis=1),
    
    columns=['norm2'],index=taste.index)
#tasteNorms
normalizedTasteWeights = pd.DataFrame(index=taste.index) #빈 데이타 프레임만들기
norms = tasteNorms['norm2']

for genre in genres.index:
    normalizedTasteWeights[genre] = taste[genre].divide(norms)
    
user_sims = pd.DataFrame(data=np.matmul(normalizedTasteWeights, normalizedTasteWeights.T))

* 아래는 유저별 유사도 그룹이다.
* 기준(standard)을 잡고 해당 기준과 평균 이상의 유사도를 가질경우 그룹으로 묶인다.

In [18]:
def p3(userlist):

    meanWeight=[]
    _sum=0
    for i in range(0,len(userlist)):
        for j in range(0,len(userlist)):     
            _sum+=user_sims[userlist[i]][userlist[j]]
        mean=float(_sum)/len(userlist)
        meanWeight.append([mean,userlist[i]])
        _sum=0
    meanWeight.sort()


    remain=userlist
    a=True
    combine=[]
    lenlist=len(meanWeight)
    while(a==True):
        for j in range(-1,-lenlist,-1):
            if meanWeight[j][1] in remain:
                standard= meanWeight[j]
                print standard,"standard"
                break
                
        __list=[]
        _remain=[]
        for i in remain:
            if user_sims[standard[1]][i] >= standard[0]-0.005:
                __list.append(i)
            else:
                _remain.append(i)
        remain=_remain  
        if len(__list)==0:
            #print "end"
            a=False
            pass
        else:
            combine.append(__list)
            
        if len(remain) ==1:
            combine.append(remain)
            remain=[]
            a=False
        elif len(remain)==0:
            a=False
    return combine



In [19]:
p3([1,2,3,4,5,6,7,8,9,10,11])

[0.8647201022006115, 1] standard
[0.860045122795858, 5] standard
[0.832599722969626, 11] standard


[[1, 2, 4, 7, 8], [3, 5, 6], [9, 11], [10]]

# 유저 정확도 테스트 방법